In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

In [ ]:
# Directories for training and validation data
train_dir = '/kaggle/input/final-dataset/Train'
val_dir = '/kaggle/input/final-dataset/Test'

In [ ]:
# Parameters
img_size = (331, 331)
batch_size = 8
epochs = 50
learning_rate = 0.0001

In [ ]:
# Data generators (no resizing since images are already the correct size)
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')


In [ ]:
# Model setup
base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(331, 331, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:

# Print model summary
print("Model Summary:")
print(model.summary())

In [ ]:
# Fine-tuning: unfreeze base model layers
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=SGD(learning_rate=learning_rate, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator)

In [ ]:
# Save the model
model_save_path = '/kaggle/working/saved_model.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# Save features for future use
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
train_features = feature_extractor.predict(train_generator, verbose=1)
val_features = feature_extractor.predict(val_generator, verbose=1)

In [ ]:
# Model Summary
model.summary()


In [ ]:
with open('/kaggle/working/train_features.pkl', 'wb') as f:
    pickle.dump(train_features, f)

with open('/kaggle/working/val_features.pkl', 'wb') as f:
    pickle.dump(val_features, f)

print("Features saved.")

In [ ]:
# Classification report
y_true = val_generator.classes
val_generator.reset()
y_pred = model.predict(val_generator, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

class_labels = list(val_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

In [ ]:
# Plot training history
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('/kaggle/working/accuracy_plot.png')
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/kaggle/working/loss_plot.png')
plt.show()

In [ ]:
# Inference demonstration
def predict_image(image_path, model, class_labels):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]

    print(f"Predicted class for {image_path}: {predicted_class}")

In [ ]:

# Example usage
sample_image_path = '/kaggle/input/final-dataset/Test/fear'
predict_image(sample_image_path, model, class_labels)
